##*Extracción, Transformación y Carga (ETL) Output_Steam_Games*

###Importando Librerias

In [142]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

###Extracción de data y Exploracion

In [143]:
#extraccion del json
row = [] #lista vacia para ir guardando las filas
with open ("output_steam_games.json") as file: #utilizo with para que el archivo se abra y cierre
    for line in file.readlines(): #bucle para ir leyendo filas y luego agregarlas a row
        data = json.loads(line)
        row.append(data)

# genero el dataframe 
games = pd.DataFrame(row)
games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [144]:
games.shape # Nos devuelve la cantidad de filas y columnas que tiene nuestro archivo

(120445, 13)

In [145]:
games.info() #Nos devuelve la cantidad de filas y columnas que tiene nuestro archivo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [146]:
# revisamos el porcentaje de filas completamente nulas respecto al total de filas del dataframe
total_filas = len(games)
filas_nulas = games.isnull().all(axis=1).sum()
porcentaje_filas_nulas = (filas_nulas / total_filas) * 100
porcentaje_filas_nulas

73.31977251027439

In [147]:
# verificamos el porcentaje de nulos por columna para tomar decisiones
total_filas = len(games)
nulos_por_columna = games.isnull().sum()
porcentaje_nulos_por_columna = (nulos_por_columna / total_filas) * 100
porcentaje_nulos_por_columna

publisher       80.004982
genres          76.045498
app_name        73.321433
title           75.021794
url             73.319773
release_date    75.035909
tags            73.455104
reviews_url     73.321433
specs           73.876043
price           74.463033
early_access    73.319773
id              73.321433
developer       76.058782
dtype: float64

La cantidad de filas con valores completamente nulos representa un 73 % del total del dataframe.


###Buscamos nulos y duplicados

In [148]:
nulos_por_columna = games.isnull().sum() # Obtenemos la cantidad de nulos por columna
nulos_por_columna

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [149]:
nulos_por_columna=games.isnull().sum()
nulos_por_columna

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [150]:
Total_nulos= games.isnull().sum().sum()
Total_nulos

1168997

In [151]:
steam_games = games.dropna(how="all").reset_index(drop=True) # Eliminamos las filas que tienen "solo valores nulos"
steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


In [152]:
total_nulos = steam_games.isnull().sum().sum() # Verificamos nuevamente la cantidad total de nulos
total_nulos

20967

Al eliminar las filas que solo tienen valores nulos, disminuimos la cantidad de los mismos de 1168997 a 20967.

In [153]:
duplicados_columnas = steam_games[steam_games.duplicated(subset=['id'])] # Verificamos si hay filas duplicadas en la columna id
duplicados_columnas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [154]:
steam_games[steam_games["developer"] == "Machine Games"]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
12237,Bethesda Softworks,[Action],Wolfenstein II: The Freedom Chronicles - Seaso...,Wolfenstein II: The Freedom Chronicles - Seaso...,http://store.steampowered.com/app/650410/Wolfe...,2017-11-07,"[Gore, Action, Violent]",http://steamcommunity.com/app/650410/reviews/?...,"[Single-player, Downloadable Content, Full con...",24.99,False,650410,Machine Games
13318,Bethesda Softworks,[Action],Wolfenstein II: The Freedom Chronicles - Episo...,Wolfenstein II: The Freedom Chronicles - Episo...,http://store.steampowered.com/app/624620/Wolfe...,2017-12-13,"[Action, Violent]",http://steamcommunity.com/app/624620/reviews/?...,"[Single-player, Downloadable Content, Steam Ac...",9.99,False,624620,Machine Games
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
28996,Bethesda Softworks,[Action],Wolfenstein: The New Order,Wolfenstein: The New Order,http://store.steampowered.com/app/201810/Wolfe...,2014-05-19,"[FPS, Action, Shooter, Alternate History, Firs...",http://steamcommunity.com/app/201810/reviews/?...,"[Single-player, Steam Achievements, Partial Co...",19.99,False,201810,Machine Games


Las filas duplicadas 14573 y 13894 son iguales por lo que podemos eliminar una de ellas.
La fila 30961 está duplicada y es idéntica a la fila 1068, pero contiene un valor nulo en el campo de identificación (ID), mientras que la fila 1068 tiene un ID completo de 2002. Por lo cual la fila duplicada 30961 se elimina.

In [155]:
steam_games = steam_games.drop([14573, 30961]) # Eliminamos columnas con id duplicadas

In [156]:
duplicados_columnas = steam_games[steam_games.duplicated(subset=['id'])]
duplicados_columnas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


In [157]:
steam_games.shape

(32133, 13)

In [158]:
# verificamos el porcentaje de nulos por columna para tomar una decision
total_filas = len(steam_games)
nulos_por_columna = steam_games.isnull().sum()
porcentaje_nulos_por_columna = (nulos_por_columna / total_filas) * 100
porcentaje_nulos_por_columna

publisher       25.058351
genres          10.216911
app_name         0.006224
title            6.379734
url              0.000000
release_date     6.432639
tags             0.507267
reviews_url      0.003112
specs            2.085084
price            4.285314
early_access     0.000000
id               0.003112
developer       10.266704
dtype: float64

La columna 'publisher' cuenta con un 25% de datos nulos siendo la de mayor cantidad.

###Transformación de los datos

In [159]:
tipos_de_datos = steam_games.dtypes # Obtenemos los tipos de datos de cada columna
tipos_de_datos

publisher       object
genres          object
app_name        object
title           object
url             object
release_date    object
tags            object
reviews_url     object
specs           object
price           object
early_access    object
id              object
developer       object
dtype: object

El conjunto de datos sigue siendo un archivo JSON, por lo que el tipo de dato es 'object', lo que podría indicar datos anidados con formato clave-valor. Se procede a analizar cada columna mediante un bucle, almacenando la información en un diccionario vacío. Posteriormente, se transforma este diccionario en un conjunto de datos para poder examinar los tipos de datos de las columnas.

In [160]:
tipo_data = {"columna":[],"tipos_de_datos":[]} # creamos un diccionario vacio

for columna in steam_games.columns: # iteramos por las columnas del dataframe 
    tipo_data["columna"].append(columna) # agregamos el nombre de la columna
    tipo_data["tipos_de_datos"].append(steam_games[columna].apply(type).unique()) # agregamos el tipo de dato de la columna

analisis= pd.DataFrame(tipo_data) # convertimos el diccionario en un dataframe
analisis

,columna,tipos_de_datos
0,publisher,"[<class 'str'>, <class 'float'>]"
1,genres,"[<class 'list'>, <class 'float'>]"
2,app_name,"[<class 'str'>, <class 'float'>]"
3,title,"[<class 'str'>, <class 'float'>]"
4,url,[<class 'str'>]
5,release_date,"[<class 'str'>, <class 'float'>]"
6,tags,"[<class 'list'>, <class 'float'>]"
7,reviews_url,"[<class 'str'>, <class 'float'>]"
8,specs,"[<class 'list'>, <class 'float'>]"
9,price,"[<class 'float'>, <class 'str'>]"


Columna "release_date"
La columna "release_date" contiene la fecha de lanzamiento de cada juego de forma YYYY-MM-DD. Transformamos esta columna de manera que solo veamos el año de lanzamiento.

In [161]:
# observamos la composición de la columna
steam_games["release_date"]

0        2018-01-04
1        2018-01-04
2        2017-07-24
3        2017-12-07
4               NaN
            ...    
32130    2018-01-04
32131    2018-01-04
32132    2018-01-04
32133    2017-09-02
32134           NaN
Name: release_date, Length: 32133, dtype: object

In [162]:
steam_games["release_date"].value_counts() # observamos la cantidad de valores por fecha

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: count, Length: 3582, dtype: int64

In [163]:
steam_games["release_date"].isnull().sum() # observamos la cantidad de nulos

2067

In [164]:
# Convertimos la columna en datetime
steam_games['release_date'] = pd.to_datetime(steam_games['release_date'], errors='coerce')

# Calculamos la fecha media
mean_date = steam_games['release_date'].mean()

# Rellenamos los valores nulos con la fecha media
steam_games['release_date'].fillna(mean_date, inplace=True)

In [165]:
# Verificamos que no hayan nulos
steam_games["release_date"].isnull().sum()

0

In [166]:
# Extraemos el año de la columna "release_date" y creamos una nueva columna llamada "release_year"
steam_games['release_year'] = steam_games['release_date'].dt.year 
steam_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018


Al contar con la columna 'release_year' optamos por eliminar 'release_date'

In [167]:
steam_games = steam_games.drop('release_date', axis=1)

###Columna "price"

In [168]:
steam_games["price"] # observamos la composición de la columna

0                4.99
1        Free To Play
2        Free to Play
3                0.99
4                2.99
             ...     
32130            1.99
32131            4.99
32132            1.99
32133            4.99
32134            4.99
Name: price, Length: 32133, dtype: object

In [169]:
steam_games["price"].value_counts() # observamos la cantidad de valores por precio

price
4.99           4278
9.99           3902
2.99           3429
0.99           2607
1.99           2541
               ... 
74.76             1
32.99             1
14.95             1
26.99             1
Free to Use       1
Name: count, Length: 162, dtype: int64

In [170]:
# Definimos una función para poder reemplazar los datos string de la columna "price" por flotantes 0.0

def str_a_float(valor):

    if pd.isna(valor): # si el valor es nulo retorno 0.0
        return 0.0
    try:
        flotante = float(valor) # si el valor es un flotante retorno el mismo valor
        return flotante
    except (ValueError, TypeError): # si el valor no es un flotante retorno 0.0
        return 0.0

In [171]:
steam_games["price"] = steam_games["price"].apply(str_a_float) # aplicamos la función a la columna "price"

In [172]:
steam_games["price"] # observamos la nueva composición de la columna

0        4.99
1        0.00
2        0.00
3        0.99
4        2.99
         ... 
32130    1.99
32131    4.99
32132    1.99
32133    4.99
32134    4.99
Name: price, Length: 32133, dtype: float64

###Columna "genres"

In [173]:
steam_games["genres"][3] # observamos la composición de un campo al azar y vemos que es una lista

['Action', 'Adventure', 'Casual']

In [174]:
steam_games["genres"].isnull().sum() # verificamos la cantidad de nulos

3283

In [175]:
steam_games["genres"].value_counts() # verificamos la cantidad de valores por genero

genres
[Action]                                                          1879
[Action, Indie]                                                   1650
[Simulation]                                                      1396
[Casual, Simulation]                                              1359
[Action, Adventure, Indie]                                        1082
                                                                  ... 
[Action, Adventure, Racing, Simulation, Strategy]                    1
[Action, Adventure, Casual, Indie, Racing, Sports, Strategy]         1
[Action, Adventure, Casual, Indie, Racing, Simulation, Sports]       1
[Action, Massively Multiplayer, RPG, Strategy]                       1
[Adventure, Casual, RPG, Simulation, Early Access]                   1
Name: count, Length: 883, dtype: int64

Aplicamos una función para limpiar la columna 'genres' y rellenar las listas con el string de índice 0 de cada lista y los valores nulos con "Pending classification"

In [176]:
def clean_genres(genres):
    if isinstance(genres, list):
        return genres[0]
    elif isinstance(genres, str):
        return genres
    else:
        return 'Pending classification'

In [177]:
# Aplicamos la función a la columna 'genres' y creamos una nueva columna 'genres_clean'
steam_games['genres_clean'] = steam_games['genres'].apply(clean_genres)

In [178]:
steam_games["genres_clean"].value_counts() # verificamos la cantidad de valores por genero

genres_clean
Action                       11316
Casual                        4340
Adventure                     4328
Indie                         3299
Pending classification        3283
Simulation                    1852
Strategy                      1143
RPG                            685
Free to Play                   523
Racing                         454
Design &amp; Illustration      327
Animation &amp; Modeling       179
Utilities                       93
Audio Production                84
Sports                          63
Massively Multiplayer           49
Education                       46
Video Production                21
Software Training               21
Web Publishing                   8
Photo Editing                    8
Accounting                       7
Early Access                     4
Name: count, dtype: int64

Observamos que 'Pending classification' ocupa el quinto lugar entre los géneros. Por esta razón, decidimos implementar una clase que contenga funciones para los géneros más conocidos. Estas funciones se encargarán de recorrer la nueva columna 'genres_clean'. En caso de encontrar 'Pending classification' en algún campo, reemplazarán ese valor con el género correspondiente encontrado en la columna 'tags'. Este enfoque nos permite evitar la pérdida de información.

In [179]:
class GenreClassifier:
    def replace_pending_with_action(self, row):
        if row['genres_clean'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Action' in row['tags']:
                    return 'Action'
        return row['genres_clean']

    def replace_pending_with_indie(self, row):
        if row['genres_clean'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Indie' in row['tags']:
                    return 'Indie'
        return row['genres_clean']

    def replace_pending_with_casual(self, row):
        if row['genres_clean'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Casual' in row['tags']:
                    return 'Casual'
        return row['genres_clean']

    def replace_pending_with_adventure(self, row):
        if row['genres_clean'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Adventure' in row['tags']:
                    return 'Adventure'
        return row['genres_clean']

    def replace_pending_with_strategy(self, row):
        if row['genres_clean'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Strategy' in row['tags']:
                    return 'Strategy'
        return row['genres_clean']

    def replace_pending_with_rpg(self, row):
        if row['genres_clean'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'RPG' in row['tags']:
                    return 'RPG'
        return row['genres_clean']

    def replace_pending_with_simulation(self, row):
        if row['genres_clean'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Simulation' in row['tags']:
                    return 'Simulation'
        return row['genres_clean']

    def replace_pending_with_racing(self, row):
        if row['genres_clean'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Racing' in row['tags']:
                    return 'Racing'
        return row['genres_clean']

    def replace_pending_with_free_to_play(self, row):
        if row['genres_clean'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Free to Play' in row['tags']:
                    return 'Free to Play'
        return row['genres_clean']

In [180]:
# Creamos una instancia de la clase
genre_classifier = GenreClassifier()

In [181]:
# Aplicamos las funciones a la columna 'genres_clean'
steam_games['genres_clean'] = steam_games.apply(genre_classifier.replace_pending_with_action, axis=1)

In [182]:
steam_games['genres_clean'] = steam_games.apply(genre_classifier.replace_pending_with_indie, axis=1)

In [183]:
steam_games['genres_clean'] = steam_games.apply(genre_classifier.replace_pending_with_casual, axis=1)

In [184]:
steam_games['genres_clean'] = steam_games.apply(genre_classifier.replace_pending_with_adventure, axis=1)

In [185]:
steam_games['genres_clean'] = steam_games.apply(genre_classifier.replace_pending_with_strategy, axis=1)

In [186]:
steam_games['genres_clean'] = steam_games.apply(genre_classifier.replace_pending_with_rpg, axis=1)

In [187]:
steam_games['genres_clean'] = steam_games.apply(genre_classifier.replace_pending_with_simulation, axis=1)

In [188]:
steam_games['genres_clean'] = steam_games.apply(genre_classifier.replace_pending_with_free_to_play, axis=1)

In [189]:
steam_games["genres_clean"].unique()

array(['Action', 'Free to Play', 'Casual', 'Adventure', 'Indie', 'Racing',
       'Pending classification', 'Strategy', 'RPG', 'Simulation',
       'Sports', 'Massively Multiplayer', 'Animation &amp; Modeling',
       'Utilities', 'Education', 'Design &amp; Illustration',
       'Audio Production', 'Video Production', 'Software Training',
       'Accounting', 'Web Publishing', 'Photo Editing', 'Early Access'],
      dtype=object)

In [190]:
steam_games["genres_clean"].value_counts()

genres_clean
Action                       12428
Casual                        4528
Adventure                     4386
Indie                         3906
Simulation                    1927
Pending classification        1207
Strategy                      1152
RPG                            693
Free to Play                   542
Racing                         454
Design &amp; Illustration      327
Animation &amp; Modeling       179
Utilities                       93
Audio Production                84
Sports                          63
Massively Multiplayer           49
Education                       46
Video Production                21
Software Training               21
Web Publishing                   8
Photo Editing                    8
Accounting                       7
Early Access                     4
Name: count, dtype: int64

In [191]:
steam_games["genres"].isnull().sum() # verificamos la cantidad de nulos de la columna "genres"

3283

In [192]:
steam_games["genres_clean"].isnull().sum() # verificamos la cantidad de nulos de la columna "genres_clean"

0

Al comparar la cantidad de nulos de las columnas 'genres y 'genres_clean', tomamos las decisión de eliminar la columna 'genres' y quedarnos con la columna 'genres_clean' que cuenta con un 100% de datos no nulos.

In [193]:
# Eliminamos la columna 'genres'
steam_games = steam_games.drop('genres', axis=1)

In [194]:
# Renombramos 'genres_clean' como 'genres'
steam_games = steam_games.rename(columns={'genres_clean': 'genres'})

In [195]:
steam_games.info() # obtenemos información de las columnas en búsqueda de nulos

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24081 non-null  object 
 1   app_name      32131 non-null  object 
 2   title         30083 non-null  object 
 3   url           32133 non-null  object 
 4   tags          31970 non-null  object 
 5   reviews_url   32132 non-null  object 
 6   specs         31463 non-null  object 
 7   price         32133 non-null  float64
 8   early_access  32133 non-null  object 
 9   id            32132 non-null  object 
 10  developer     28834 non-null  object 
 11  release_year  32133 non-null  int32  
 12  genres        32133 non-null  object 
dtypes: float64(1), int32(1), object(11)
memory usage: 4.3+ MB


In [196]:
total_filas = len(steam_games)
nulos_por_columna = steam_games.isnull().sum()
porcentaje_nulos_por_columna = (nulos_por_columna / total_filas) * 100
porcentaje_nulos_por_columna

publisher       25.058351
app_name         0.006224
title            6.379734
url              0.000000
tags             0.507267
reviews_url      0.003112
specs            2.085084
price            0.000000
early_access     0.000000
id               0.003112
developer       10.266704
release_year     0.000000
genres           0.000000
dtype: float64

Por último y para finalizar el trabajo en esta columna y ya habiendo cumplido con su propósito, eliminamos la columna 'tags'.

In [197]:
#Eliminamos la columna 'tags'
steam_games = steam_games.drop('tags', axis=1)

Columna 'specs'
Esta columna, almacena información sobre especificaciones técnicas de los juegos, tales como si es "Single-player", "Multi-player", "Co-op", etc., haciendo que la misma no sea de mucha utilidad para nuestro análisis. Por esta razón, decidimos eliminarla.

In [198]:
steam_games = steam_games.drop('specs', axis=1)

Columnas "publisher" y "developer"
Apreciamos que las columnas 'publisher' y 'developer' cuentan con un 25% y 10% de datos nulos respectivamente. Por esta razón, y en función de que suelen estar directamente relacionadas, optamos por aplicar una funcion que nos permita rellenar los valores nulos de la columna 'developer' con los valores coincidentes en el mismo campo de la columna 'publisher'.

In [199]:
def fill_null_developers_with_publisher(row):
    if pd.isnull(row['developer']):
        return row['publisher']
    return row['developer']

# AplicaMOS la función a la columna 'developer'
steam_games['developer'] = steam_games.apply(fill_null_developers_with_publisher, axis=1)

In [200]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 0 to 32134
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24081 non-null  object 
 1   app_name      32131 non-null  object 
 2   title         30083 non-null  object 
 3   url           32133 non-null  object 
 4   reviews_url   32132 non-null  object 
 5   price         32133 non-null  float64
 6   early_access  32133 non-null  object 
 7   id            32132 non-null  object 
 8   developer     28899 non-null  object 
 9   release_year  32133 non-null  int32  
 10  genres        32133 non-null  object 
dtypes: float64(1), int32(1), object(9)
memory usage: 3.8+ MB


Teniendo en cuenta que las consignas de funciones nos piden trabajar con DESARROLLADORES, optamos por eliminar la columna 'publisher' y quedarnos con la columna developer.

In [201]:
# Eliminamos la columna 'publisher'
steam_games = steam_games.drop('publisher', axis=1)

Basándonos en un análisis previo y para evitar el sesgo que podría generar rellenar los campos nulos de la columna developer con 'No Data', siendo esta columna de mucha importancia para los endpoints. Decidimos eliminar las filas que contengan valores nulos en la columna developer.

In [202]:
# Filtramos y mantenemos solo las filas donde 'developer' no sea nulo
steam_games = steam_games[steam_games['developer'].notnull()]

# Reestablecemos los índices despúes de filtrar
steam_games = steam_games.reset_index(drop=True)

###Columna "id"

In [203]:
# Rellenamos los valores NaN en la columna 'id' con 'No Data'
steam_games['id'] = steam_games['id'].fillna('No Data')

# Convertimos la columna 'id' a tipo string
steam_games['id'] = steam_games['id'].astype(str)

In [204]:
# Renombramos la columna 'id' como 'item_id'
steam_games = steam_games.rename(columns={'id': 'item_id'})

In [205]:
# Verificamos la información de las columnas
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28899 entries, 0 to 28898
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   app_name      28899 non-null  object 
 1   title         28899 non-null  object 
 2   url           28899 non-null  object 
 3   reviews_url   28899 non-null  object 
 4   price         28899 non-null  float64
 5   early_access  28899 non-null  object 
 6   item_id       28899 non-null  object 
 7   developer     28899 non-null  object 
 8   release_year  28899 non-null  int32  
 9   genres        28899 non-null  object 
dtypes: float64(1), int32(1), object(8)
memory usage: 2.1+ MB


###Columnas "early_access", "url" y "review_url"

Estas columnas no aportan información relevante para nuestro análisis, por lo que decidimos eliminarlas.

In [206]:
# Eliminamos las columnas 'early_access', 'url' y 'reviews_url'
steam_games = steam_games.drop(['early_access', 'url', 'reviews_url'], axis=1)

Columnas "app_name" y "title"
Las columnas 'app_name' y 'title' cuentan con un 0.006% y 6.37% de datos nulos respectivamente. Por esta razón, y en función de que estan directamente relacionadas, optamos por eliminar la segunda.

In [207]:
# Eliminamos la columna 'title'
steam_games = steam_games.drop('title', axis=1)
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28899 entries, 0 to 28898
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   app_name      28899 non-null  object 
 1   price         28899 non-null  float64
 2   item_id       28899 non-null  object 
 3   developer     28899 non-null  object 
 4   release_year  28899 non-null  int32  
 5   genres        28899 non-null  object 
dtypes: float64(1), int32(1), object(4)
memory usage: 1.2+ MB


###Guardamos el dataset como CSV

In [208]:
steam_games.to_csv("steam_games.csv", index=False, encoding="utf-8")

Hemos optado por convertir el archivo CSV al formato Parquet con el fin de optimizar el almacenamiento y mejorar la eficiencia en las operaciones de lectura y escritura. La elección de la transformación a Parquet se fundamenta en su facilidad para manejar conjuntos de datos complejos. Esta conversión facilita el procesamiento distribuido y ofrece opciones avanzadas de compresión, lo que contribuye a una gestión más eficiente de los recursos y a un mejor rendimiento en entornos de procesamiento de datos.

In [209]:
steam_games = pd.read_csv("steam_games.csv") # leemos el archivo csv

tabla = pa.Table.from_pandas(steam_games) # convertimos el dataframe en una tabla
pq.write_table(tabla,"steam_games.parquet") # guardamos la tabla en un archivo parquet